In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_valid,y_valid) = mnist.load_data()

def plot_image(data, idx):
    plt.figure(figsize = (5,5))
    plt.imshow(data[idx], cmap = 'gray')
    plt.axis('off')
    plt.show()
plot_image(x_train, 0)

In [ ]:
print(x_train.min(), x_train.max())
print(x_valid.min(), x_valid.max())

In [ ]:
x_train = x_train/x_train.max()
x_valid = x_valid/x_valid.max()

In [ ]:
print(x_train.min(), x_train.max())
print(x_valid.min(), x_valid.max())

In [ ]:
x_train.shape

In [ ]:
x_train_in = x_train[..., tf.newaxis]
x_valid_in = x_valid[..., tf.newaxis]
print(x_train_in.shape, x_valid_in.shape)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (28,28,1),name = 'conv'),
    tf.keras.layers.MaxPool2D((2,2), name = 'pool'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10,activation = 'softmax')
])

model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train,y_train, validation_data = (x_valid, y_valid), epochs = 10)

In [ ]:
model.summary()

In [ ]:
model.evaluate(x_valid_in, y_valid)

In [ ]:
loss, val_loss = history.history['loss'], history.history['val_loss']
acc, val_acc = history.history['accuracy'], history.history['val_accuracy']
epoch = 10
fig, axes = plt.subplots(1,2,figsize = (12,4))
axes[0].plot(range(1, epoch+1), loss, label = 'Training')
axes[0].plot(range(1, epoch+1), val_loss, label = 'Validation')
axes[0].legend(loc = 'best')
axes[0].set_title('Loss')

axes[1].plot(range(1, epoch+1), acc, label = 'Training')
axes[1].plot(range(1, epoch+1), val_acc, label = 'Validation')
axes[1].legend(loc = 'best')
axes[1].set_title('Accuracy')

plt.show()

In [ ]:
model.layers

In [ ]:
model.layers[0].weights

In [ ]:
model.layers[0].bias

In [ ]:
model.layers[0].kernel

In [ ]:
model.layers[0].bias

In [ ]:
model.get_layer('conv')

In [ ]:
activator = tf.keras.Model(inputs=model.input,outputs = [layer.output for layer in model.layers[:2]])

In [ ]:
activations = activator.predict(x_train_in[0][tf.newaxis, ...])
len(activations)

In [ ]:
conv = activations[0]
conv.shape

In [ ]:
fig, axes = plt.subplots(4,8)
fig.set_size_inches(10, 5)

for i in range(32):
    ax = axes[i//8, i%8]
    ax.imshow(conv[0, :, :, i], cmap = 'viridis')
    ax.set_title('kernel %s'%str(i), fontsize=10)
    plt.setp( axes[i//8, i%8].get_xticklabels(), visible=False)
    plt.setp( axes[i//8, i%8].get_yticklabels(), visible=False)

plt.tight_layout()
plt.show()

In [ ]:
pooling = activations[1]
pooling.shape

In [ ]:
fig, axes = plt.subplots(4,8)
fig.set_size_inches(10, 5)

for i in range(32):
    ax = axes[i//8, i%8]
    ax.imshow(conv[0, :, :, i], cmap = 'viridis')
    ax.set_title('kernel %s'%str(i), fontsize=10)
    plt.setp( axes[i//8, i%8].get_xticklabels(), visible=False)
    plt.setp( axes[i//8, i%8].get_yticklabels(), visible=False)

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
import tensorflow_hub as tfhub

In [ ]:
img_path = 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c4/Gangnam_Seoul_January_2009.jpg/1280px-Gangnam_Seoul_January_2009.jpg'
img = tf.keras.utils.get_file(fname='gangnam', origin = img_path)
img = tf.io.read_file(img) 
img = tf.image.decode_jpeg(img, channels=3)
img = tf.image.convert_image_dtype(img, tf.float32) 

import matplotlib.pylab as plt
plt.figure(figsize=(15, 10))
plt.imshow(img) 

In [ ]:
img.shape

In [ ]:
img_input = tf.expand_dims(img, 0)
img_input.shape

In [ ]:
#드라이브에 넣어도 안올라가지고 이미지 긁어와도 안올라가네요 뭐지????
model = tfhub.load("https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.signatures.keys()

In [ ]:
obj_detector = model.signatures['default']
obj_detector

In [ ]:
result = obj_detector(img_input)
result.keys()

In [ ]:
type(result)

In [ ]:
labels = result["detection_class_labels"]
names = result['detection_class_names']

In [ ]:
len(result["detection_scores"])

In [ ]:
result.keys()

In [ ]:
for key, value in result.items():
    print(key, value)

In [ ]:
img_height, img_width = img.shape[0], img.shape[1]
obj_to_detect = 10
for i in range(min(obj_to_detect, boxes.shape[0])):
    if scores[i] >= 0.2:
        (ymax, xmin, ymin, xmax) = (boxes[i][0]*img_height, boxes[i][1]*img_width,
                                    boxes[i][2]*img_height, boxes[i][3]*img_width)
        print("{} : ({},{},{},{})". format(scores[i], ymax, xmin, ymin, xmax))

In [ ]:
boxes = result["detection_boxes"] 
labels = result["detection_class_entities"]
scores = result["detection_scores"]

img_height, img_width = img.shape[0], img.shape[1]

obj_to_detect = 10

plt.figure(figsize = (15, 10))
for i in range(min(obj_to_detect, boxes.shape[0])):
    if scores[i] >= 0.2:
        (ymax, xmin, ymin, xmax) = (boxes[i][0]*img_height, boxes[i][1]*img_width,boxes[i][2]*img_height, boxes[i][3]*img_width)
        
        plt.imshow(img)
        plt.plot([xmin, xmax, xmax, xmin, xmin], [ymin, ymin, ymax, ymax, ymin],color = 'yellow', linewidth=2)
        
        class_name = labels[i].numpy().decode('utf-8')
        infer_score = str(scores[i].numpy()*100)
        annotation = "{}: {}%", format(class_name, infer_score)
        plt.text(xmin+10, ymax+20, annotation,color='white', backgroundcolor='blue', fontsize=10)
     